In [266]:
import pandas as pd
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

#### 데이터 불러오기
#### hanok - DataFrame
#### hanok2 - Series
#### sukso_merge - DataFrame
#### susko_merge_list - Series

In [267]:
hanok = pd.read_csv('한옥스테이.csv')
sukso_recommendation = pd.read_csv('숙소추천.csv')
sukso = pd.read_csv('숙소.csv')

한옥 데이터 처리

In [268]:
hanok2 = pd.read_csv('한옥스테이.csv')

In [269]:
hanok2.drop('Unnamed: 0', axis = 1)

,id,date,like,text,hashtag,src,target
0,jrksh512,2021-05-07 00:00:00+00:00,487,홍보용으로 쓰라고 손님이 보내준 고마운 사진 7주,"['#지례예술촌', '#안동숙소', '#안동숙박', '#한옥스테이', '#힐링숙소'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
1,j_chaeny,2021-05-16 00:00:00+00:00,142,_ 딸맘의 로망 이루며 사는중 🥰 (Feat.외로운 사진기사 아빠) 6주,"['#여행끝', '#사진은많이남음']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
2,chungju_travel,2021-05-12 00:00:00+00:00,465,⠀ 충주에 윤스테이감성 뿜뿜하는 한옥스테이가 있다고?👀 ⠀ 고즈넉한 연못 위의 찻방...,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
3,hyevvon__,2021-05-08 00:00:00+00:00,507,저녁에 빗소리 들으며 마셨던 맥주와 화창한 아침에 햇살 받으며 먹은 조식은 못 잊을...,"['#여행', '#국내여행', '#충주', '#충주여행', '#한옥', '#한옥펜션...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
4,golf_sra,2021-05-22 00:00:00+00:00,384,여름 문턱의 쌍산재🌿 5주,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
...,...,...,...,...,...,...,...
6417,byulhenun,2021-03-04 00:00:00+00:00,48,새로운 시작에 바쁜 요즘이예요🤯 주말엔 아무런 방해도 받지 말고 평화로운 풍경 속에...,"['#사촌마을', '#전통주안상', '#전통주', '#의성여행', '#ᄅ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
6418,ahem.jpg,2021-03-04 00:00:00+00:00,130,☀️ 햇살 맛집 덕분에 포근했던 1박2일 16주,"['#하은재', '#한옥스테이']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
6419,seo_yeong_star,2021-03-04 00:00:00+00:00,151,"스테이황룡에는 긴 연못이 있어요~ 이렇게 비오는날 빗방울이 하나,둘 떨어지고 있는 ...","['#스테이황룡', '#비오는날', '#연못', '#운치스타그램', '#한옥스타그램...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1
6420,flrou82018,2021-03-04 00:00:00+00:00,60,노랭이 길냥이가와서 아침마다 야옹야옹하며 밥달라고 하네요 16주,"['#전주한옥마을한옥독채', '#구강재', '#한옥체험숙소', '#한옥뷔가멋진집',...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,1


In [270]:
index_del = hanok[hanok['hashtag'] == '[]'].index
hanok = hanok.drop(index_del)

In [271]:
hanok_list = [] # 비어있는 리스트 만들기 
for txt in hanok['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#","")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    
    hanok_list.append(del_br) #값 채우기

In [272]:
hanok_list= pd.Series(hanok_list)

In [273]:
hanok_list

0          지례예술촌 안동숙소 안동숙박 한옥스테이 힐링숙소 힐링여행 국내여행 감성사진 고택체험
1                                             여행끝 사진은많이남음
2       여행 국내여행 충주 충주여행 한옥 한옥펜션 한옥스테이 찐힐링 명소 핫플 여행그램 서...
3                                                서산 한옥스테이
4       강릉김씨네탱자나무집 강김탱 한옥 강릉 강릉여행 강릉숙소 한옥스테이 독채 독채펜션 바...
                              ...                        
5488    산수유마을 트렁크마켓 산수유사랑채 이천숙소 한옥스테이 이천가볼만한곳 산수유꽃 바랑골...
5489    사촌마을 전통주안상 전통주 의성여행 랜선여행 국내여행 경북여...
5490                                            하은재 한옥스테이
5491    스테이황룡 비오는날 연못 운치스타그램 한옥스타그램 한옥스테이 한옥게스트하우스 장성핫...
5492            전주한옥마을한옥독채 구강재 한옥체험숙소 한옥뷔가멋진집 한옥스테이 전주 여행
Length: 5493, dtype: object

In [274]:
hanok = hanok_list.str.contains("숙소") == 0
hanok = hanok_list.str.contains("숙소추천") == 0
hanok = hanok_list[hanok]

In [275]:
hanok2 = hanok2[hanok2['hashtag'].str.contains("숙소") ==0]
hanok2 = hanok2[hanok2['hashtag'].str.contains("숙소추천") ==0]

In [276]:
hanok2 = hanok2.drop('Unnamed: 0', axis = 1)

In [277]:
hanok2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4586 entries, 1 to 6421
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       4586 non-null   object
 1   date     4586 non-null   object
 2   like     4586 non-null   object
 3   text     4586 non-null   object
 4   hashtag  4586 non-null   object
 5   src      4328 non-null   object
 6   target   4586 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 286.6+ KB


In [278]:
index_del = hanok2[hanok2['hashtag'] == '[]'].index
hanok2 = hanok2.drop(index_del)

In [279]:
for i in range(len(hanok)):
    for j in range(len(hanok.iloc[i].split(","))) :        
        print(hanok.iloc[i].split(",")[j])

지례예술촌 안동숙소 안동숙박 한옥스테이 힐링숙소 힐링여행 국내여행 감성사진 고택체험
여행끝 사진은많이남음
여행 국내여행 충주 충주여행 한옥 한옥펜션 한옥스테이 찐힐링 명소 핫플 여행그램 서유숙펜션 펜션추천 충주펜션 조식맛집 travel trip ff likesforlike
서산 한옥스테이
강릉김씨네탱자나무집 강김탱 한옥 강릉 강릉여행 강릉숙소 한옥스테이 독채 독채펜션 바베큐 바베큐그릴 가스바베큐
일엽편주 농암종택 한옥 한옥스테이 안동 일엽편주 안동하회마을 여름휴가 주말여행 andong travelgram ellyeoppyunjoo ricewine hanok koreanheritage travelkorea summerlytaste_andong summerlytaste_korea
남원 스테이리운 스테이폴리오 스테이폴리오_전라 남원 한옥스테이
한옥스테이 일독일박 47개월하짱이 서촌가볼만한곳 한옥체험 아이와여행 아이와갈만한곳
전주오늘도묘한밤 한옥마을게스트하우스
셔니네민박 전주 객리단길 한옥스테이 동가스테이 연식당 서양주택 지켜츄
1박2일 한옥스테이 아그니 LP방 30년지기 MUR친구
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐링 여행 완주 전주 전라도 오성제
청원당 한옥스테이 한옥 청송 고택 펜션 민박 호텔 조식 조식맛집 한옥마을 윤스테이 관광 다문화 수업 풍경 게스트하우스 다도체험 다도 꽃 한옥체험 게스트룸 차 茶 korea traditional hanok guesthouse breakfast teaceremony
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐링 여행 완주 전주 전라도 조식
공주한옥 게스트하우스 공주 한옥 한옥게스트하우스 국내여행 공주여행 힐링 봉황재 감성 여행스타그램 한옥스테이 스냅 일상 리모트워크 레트로 빈티지 한옥마을 마을스테이 굿즈 마을호텔 korea travel koreatravel hanok gongju travelgram traveltheworld
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐

완주 아원고택 아원 한옥스테이 아메리카노
더프레임tv 이호소락 한옥 인테리어 한옥스테이 한옥호텔 한옥체험 북촌한옥 국립현대미술관 삼청동 인사동 여행 hanokstay hanok hanokhotel koreahouse koreatravel insadong samchengdong korea hotel airbnb airbnbseoul
덕선재 Canon_prima_zoom85n filmphoto
봉황재 봉황재한옥 공주한옥 게스트하우스 공주 한옥 공주숙소 국내여행 공주여행 힐링 봉황재 여행스타그램 한옥스테이 스냅 사은품 리모트워크 레트로 마을스테이 제민천 마을호텔 리그램이벤트 이벤트 korea travel koreatravel hanok gongju regram traveltheworld
올모스트홈 솜씨여행
더프레임tv 이호소락 한옥 인테리어 한옥스테이 한옥호텔 한옥체험 북촌한옥 국립현대미술관 삼청동 인사동 여행 hanokstay hanok hanokhotel koreahouse koreatravel insadong samchengdong korea hotel airbnb airbnbseoul
시골집수리 한옥리모델링 셀프공사 농가주택수리 촌집수리 감성한옥숙소 스테이보름 경주숙소 감성숙소 한옥스테이 경주여행 셀프인테리어 헌집줄게새집다오 경주살이 시골살이 인테리어디자인 한옥인테리어 스타코 스타코미장 스타코뿜칠 페인트 셀프페인트칠
강릉여행 여행스타그램 강릉숙소 한옥스테이 윤스테이 감성숙소 감성숙소_강릉 독채팬션 소통해요 몰튼브라운 몰튼브라운어메니티 욕조인테리어
봉황재 봉황재한옥 공주한옥 게스트하우스 공주 한옥 공주숙소 국내여행 공주여행 힐링 봉황재 여행스타그램 한옥스테이 스냅 사은품 리모트워크 레트로 마을스테이 제민천 마을호텔 리그램이벤트 이벤트 

소소가 경주숙소 한옥펜션 한옥민박 경주한옥펜션 황리단길 저녁밥 소소가밥상 소소가펜션 경주여행 가족여행 경주가족여행 여행스타그램 한옥스테이 밥주는민박 밥상스타그램
벚꽃 감성사진 카페투어 헨드메이드 인테리어소품 인테리어 캔들 향브랜드 필로우미스트 향초 한옥스테이 대구여행 뭉향 일상
덕수당한옥펜션 전남 함평 광주 주포한옥마을 덕수당한옥펜션 덕수당 한옥스테이 광주근교나들이 광주근교한옥 광주근교여행 펜션 펜션추천 한옥펜션 한옥펜션추천 덕수당 함평주포한옥마을 함평주포한옥마을펜션 가족모임 가족모임장소 친구모임장소 칠순잔치장소 파티모임장소 겨울 바베큐파티 야외 단체모임환영 단체모임장소 함평가볼만한곳
춘천_지혜원 감성숙소_춘천 감성숙소_강원도 강원도 춘천 지혜원 춘천여행 춘천펜션 춘천한옥펜션 춘천독채펜션 강원도펜션 강원도여행 강원도독채펜션 감성숙소 독채펜션 한옥스테이 에어비앤비 춘천에어비앤비
명주하녹 한옥스테이 강릉독채 한옥리모델링 한옥숙소 강릉여행 강릉숙소 강릉에어비앤비 프라이빗숙소 강릉독채스테이 hanokstay
대구여행 제이비관광호텔 한옥호텔 개량한복 퓨즈서울 굿초여행 굿초호텔
osohanok 오소한옥 바름26 varm26
청원당 한옥스테이 한옥 청송 고택 펜션 민박 호텔 조식 조식맛집 한옥마을 윤스테이 관광 프랑스자수 도예 게스트하우스 다도체험 다도 꽃 한옥체험 게스트룸 차 茶 netherlands korea traditional hanok guesthouse breakfast teac

In [280]:
for i in range(len(hanok)):
    print(hanok.iloc[i])

지례예술촌 안동숙소 안동숙박 한옥스테이 힐링숙소 힐링여행 국내여행 감성사진 고택체험
여행끝 사진은많이남음
여행 국내여행 충주 충주여행 한옥 한옥펜션 한옥스테이 찐힐링 명소 핫플 여행그램 서유숙펜션 펜션추천 충주펜션 조식맛집 travel trip ff likesforlike
서산 한옥스테이
강릉김씨네탱자나무집 강김탱 한옥 강릉 강릉여행 강릉숙소 한옥스테이 독채 독채펜션 바베큐 바베큐그릴 가스바베큐
일엽편주 농암종택 한옥 한옥스테이 안동 일엽편주 안동하회마을 여름휴가 주말여행 andong travelgram ellyeoppyunjoo ricewine hanok koreanheritage travelkorea summerlytaste_andong summerlytaste_korea
남원 스테이리운 스테이폴리오 스테이폴리오_전라 남원 한옥스테이
한옥스테이 일독일박 47개월하짱이 서촌가볼만한곳 한옥체험 아이와여행 아이와갈만한곳
전주오늘도묘한밤 한옥마을게스트하우스
셔니네민박 전주 객리단길 한옥스테이 동가스테이 연식당 서양주택 지켜츄
1박2일 한옥스테이 아그니 LP방 30년지기 MUR친구
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐링 여행 완주 전주 전라도 오성제
청원당 한옥스테이 한옥 청송 고택 펜션 민박 호텔 조식 조식맛집 한옥마을 윤스테이 관광 다문화 수업 풍경 게스트하우스 다도체험 다도 꽃 한옥체험 게스트룸 차 茶 korea traditional hanok guesthouse breakfast teaceremony
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐링 여행 완주 전주 전라도 조식
공주한옥 게스트하우스 공주 한옥 한옥게스트하우스 국내여행 공주여행 힐링 봉황재 감성 여행스타그램 한옥스테이 스냅 일상 리모트워크 레트로 빈티지 한옥마을 마을스테이 굿즈 마을호텔 korea travel koreatravel hanok gongju travelgram traveltheworld
소양고택 두베카페 플리커책방 한옥스테이 완주여행 전주여행 힐

장성한옥 한옥스테이 게스트하우스 한옥체험 장성핫플 한옥카페 전통한옥 한옥 펜션 한옥펜션 장성펜션 담양펜션 부안펜션 캠핑 오토캠핑 광주캠핑장 전라도가볼만한곳 민박 음료 식사 Hanok guesthouse motel drink meal brunch dinner supper
스테이넉넉
아들생일 한옥스테이 아이와함께 아들스타그램 딸스타그램 엄마사람
한옥스테이 보성 한옥 여행 여행스타그램 데일리 일상스타그램 일상 휴일 전라남도 전남여행
도어락따위 아침 굿모닝 마루 한옥스테이 전주 전주여행 전주한옥마을 여행 여행스타그램
이상한일상 전주 로컬프로젝트
명주하녹 강릉 한옥스테이 강릉여행 강릉숙소 강릉스테이 강릉독채 프라이빗숙소 독채펜션 hanokstay 강원도숙소 강원도여행 스테이폴리오 stayfolio 스테이폴리오_강릉 스테이폴리오_강원 강릉펜션 강원도펜션 강릉에어비앤비 에어비앤비_강릉 명주동 시나미명주
오늘우리집 한옥스테이 우리셋 생일주간
청연재
청연재
서울이야기한옥 게하스타그램 서울게스트하우스 서울한옥숙박 서울가족숙소 서울한옥독채 한옥스테이 한옥체험 서울에어비앤비 에어비앤비서울 서울한옥마을 서울갈만한곳 서울나들이 서울근교여행 급여행지추천 가족여행지추천 국내여행지추천 서울파티룸숙박 일상소통 맞팔 좋반 좋아요 일상을여행으로 일상그램 여행그램 인사동 익선동 삼청동 안국역
물놀이 놀러오세요 서울이야기한옥 서울게스트하우스 서울한옥숙박 서울가족숙소 서울한옥독채 한옥스테이 한옥체험 서울에어비앤비 에어비앤비서울 서울한옥마을 서울갈만한곳 서울나들이 서울근교여행 급여행지추천 가족여행지추천 국내여행지추천 서울여행 일상소통 맞팔 좋반 좋아요 서울풀빌라 서울가족호텔 일상을여행으로 일상그램 여행그램 인사동 삼청동
k갬성 광화문 야경맛집 경복궁 서울여행 서울이야기한옥 서울게스트하우스 서울한옥숙박 서울가족숙소 서울한옥독채 한옥스테이 한옥체험 서울에어비앤비 에어비앤비서울 서울한옥마을 서울갈만한곳 서울나들이 서울근교여행 급여행지추천 가족여행지추천 국내여행지추천 서울파티룸숙박 일상소통 맞팔 좋반 좋아요 

한옥풀빌라 한옥수영장 하루한팀 한옥스테이온 스테이온 한옥스테이 경주한옥펜션 경주독채펜션 경주한옥 한옥펜션 독채한옥 1일1팀 경주한옥풀빌라
담소 전주 한옥마을 힐링 여행 여유 아메리카노 일상 예약문의 숙소 숙박 한옥스테이
지례예술촌 안동숙박 안동숙소 안동여행 힐링숙소 고택체험 국내여행 한옥스테이 경북숙소 경북여행
안동여행 군자마을 한옥스테이 여행의즐거움 늘해랑아뜰리에공방 동탄플라워공방 동탄프리저브드공방 한옥체험
아소재 한옥체험 한옥스테이 한옥카페 텃밭채소
어제그램 전주한옥마을 첫장거리가족여행 당충전 카페인충전 자식이뭐길래 엄마도그랬어 가족여행 전주여행 한옥스테이 한옥마을 주말여행
전주오늘도묘한밤 한옥마을게스트하우스
우리것이좋은것이여 한옥스테이 힐링숙소 모닝뷰맛집 충전중🔌
한옥스테이
첫줄 한옥스테이 달빛향 전주 전주한옥마을 좋반 여행 소소한행복 맞팔 선팔
koreantraditionalhouse koreantraditionaldress 한옥 한복 그리스아기아씨나 한복사진 한옥스테이 한옥펜션 단산 문경 주말여행 강아지스타그램 단산한옥펜션 hanok hanokvillage hanbok
함양정일품명가 함양여행 스몰웨딩 야외웨딩 한옥스테이 한옥체험 한옥펜션 피네웨딩 여행그램 koreatravel hanokstay hanokvillage traveler wedding instawedding
경주여행 한옥스테이 오릉한옥 오릉한옥게스트하우스 조식스타그램 조식펜션 한옥건축
로투스쿠키 얼그레이무화과스콘
한옥카페 한옥마을 한옥스테이 한옥 카페추천 카페투어 한옥마을 맞팔100 맞팔해요우리 소통 소통해요 소통해요우리 주말일상 주말시작 일상그램 일상소통 육아소통 육아그램 인친그램 인친 일상 데일리 좋아요반사 좋아요 좋반 좋반테러 힐링 맞팔 피드 토요일
한옥스테이 통영숙소 탱란부부 가족여행
완주 두베카페 소양고택 플리커책방 소양고택한옥스테이 두베카페 오성한옥마을 완주 완주여행 여행스타그램 소양고택 두베카페 플리커책방 오성한옥마을 힐링스팟 조용한여행 한옥감성 한옥스테이 완주카페 완주가볼만한

더채갤러리 스테이폴리오_서울 서촌 한옥스테이
문화객가사랑채 고택 밀양고택 한옥 한옥스테이 한옥체험 밀양 밀양문화재 밀양여행 여행 밀양숙소 숙소 밀양향교 밀양한달살이 airbnb 에어비앤비
문화객가사랑채 고택 밀양고택 한옥 한옥스테이 한옥체험 밀양 밀양문화재 밀양여행 여행 밀양숙소 숙소 밀양향교 밀양한달살이 airbnb 에어비앤비
서촌한옥 한옥 한옥스테이 더채라이브러리 thechae
더채 thechae 한옥스테이 hanok
세종공방 세종도자기체험 세종도자기공방 한옥펜션추천 한옥카페 한옥스테이 한옥호텔 한옥숙소 휴휴당 세종휴휴당
한옥리모델링 대전디자인스튜디오 한옥카페 한옥스테이 한옥스테이인테리어 구옥개조 대전카페 대전카페인테리어
굿모닝 좋은하루되세요 햇살가득 햇살맛집 스테이황룡 사랑채 한옥스타그램 한옥스테이 한옥게스트하우스 한옥인테리어 장성핫플 담양핫플 광주핫플 숙소 민박 펜션 담양펜션 함평숙소 전남펜션 감성숙소 광주펜션 광주민박 장성민박 장성펜션 전남여행 전남감성숙소
한옥스테이 달빛한옥마을 여락재
아원고택 전주 완주 한옥스테이 여행스타그램
빔프로젝터 킹스빔 영화관이따로없네 치맥타임 스테이황룡 사랑채1 한옥스타그램 한옥스테이 한옥게스트하우스 한옥인테리어 장성핫플 담양핫플 광주핫플 숙소 민박 펜션 담양펜션 함평숙소 전남펜션 감성숙소 광주펜션 광주민박 장성민박 장성펜션 전남여행 전남감성숙소
어바웃돌_자명서실
어바웃돌_자명서실
어바웃돌_자명서실
어바웃돌_자명서실
어바웃돌_자명서실
어바웃돌_자명서실
아원고택 가족여행 한옥스테이 완주
도정서원 한옥스테이 예천여행🚘
가족여행 아원고택 한옥스테이 완주
죽녹원 영빈관 담양호텔 담양숙소 한옥체험 한옥스테이 취죽헌 🛌 마당에서 보이는 대나무숲 👍 온돌 바닥이 뜨거워서 익을뻔 ♥️
인테리어 서울인테리어 디자인모리 주택인테리어 구옥리모델링 구옥인테리어 대수선 구옥 한옥개조 한옥인테리어 한옥리모델링 한옥 한옥스테이 한옥팬션 

In [281]:
hanok2.reset_index(drop = True, inplace = True)

In [282]:
hanok.reset_index(drop = True, inplace = True)

In [283]:
hanok2['hashtag'] = hanok

In [284]:
hanok_data = hanok2.iloc[:, [4,6]]

In [285]:
# hanok_X = hanok2.iloc[:, [4]]
# hanok_y = hanok2.iloc[:, [6]]

In [286]:
len(hanok_y)

3657

# 숙소 데이터 처리

숙소 - 숙소추천 해줌으로써 중복된 값 제거

In [287]:
sukso = sukso[sukso['hashtag'].str.contains('숙소추천')==0]

In [288]:
sukso

,id,date,like,text,hashtag,src,target
0,cheonyonghyun,2021-06-14 00:00:00+00:00,637,🙏🏻 ⠀ 여행에서 숙소가 주는 소소한 행복 ⠀ 2020년 9월 태풍 마이삭으로 인해...,"['#보름우스테이', '#제주도', '#제주여행', '#제주숙소', '#숙소', '...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
1,16.2yc,2021-04-29 00:00:00+00:00,498,오늘 날씨 칭찬해🏄‍♀️🏄‍♀️ 8주,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
2,90_hj_,2021-05-21 00:00:00+00:00,115,"진짜 오빠랑 나의 인생숙소, 우연히 작년 오픈초반에 알게되어 다녀왔는데 너무 조용하...","['#숙소', '#영덕숙소', '#영덕풀빌라', '#해온안', '#인생숙소', '#...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,with_sn,2021-05-02 00:00:00+00:00,"1,887","- 자기 생일을 제일 쑥스러워하는 편😅 5월은 부모님 생신이며 어버이날, 경나와의 ...","['#기대도실망도없는', '#행복한생일주간😘']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
5,with_sn,2021-05-04 00:00:00+00:00,729,- 펜션 호텔만 가면 지루해서 잠만 자다 오기 바쁜 너인데😅 얼마나 마음에 들었는지...,"['#그래서오늘도나는', '#너의사진첩을뒤져본다']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
6381,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,"['#누마루', '#파란하늘', '#방문자리뷰', '#스테이황룡', '#한옥스타그램...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6382,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,"['#정방폭포', '#제주도', '#황사', '#신행', '#신혼여행', '#숙소']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6383,minor_ri,2021-03-30 00:00:00+00:00,381,날씨 엄청 좋았었는데 여행가고싶은 요즘😢💙 13주,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6385,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,"['#마지막', '#기록', '#제주', '#여행', '#뚜벅이', '#혼여행', ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


숙소, 숙소추천 데이터 결합

In [289]:
sukso_merge = pd.concat([sukso_recommendation,sukso] , axis= 0)

In [290]:
sukso_merge

,id,date,like,text,hashtag,src,target
0,s2kyung,2021-06-15 00:00:00+00:00,457,특별한 제주여행을 위해 고민하다 선택한 첫 번째 숙소는 우도에 위치한 스테이소도💛 ...,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
1,travelmenu_stay,2021-06-23 00:00:00+00:00,"2,400",[ 널찍한 통창뷰로 자연을 온전히 즐길 수 있는 곳🌿💚 ] 📍30만원대 초반(평일 ...,"['#춘천', '#수안', '#여행머무름', '#여름_춘천', '#여행머무름_춘천'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
2,busansomang,2021-06-21 00:00:00+00:00,"2,086","감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...","['#제주', '#제주', '#제주도', '#숙소', '#제주숙소', '#숙소추천'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,zzazang_,2021-06-26 00:00:00+00:00,"3,385",이거 하나보고 예약했던 숙소 🪨🪵🐷🥩🥬🍹💖 . . . 2일,"['#제주여행', '#제주도여행', '#제주도', '#제주', '#제주숙소', '#...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
4,ejiyang,2021-06-09 00:00:00+00:00,"1,785",문의주신 숙소 공유해드려요♥️ 서귀포에 위치한 이에요🌼 이번 제주는 서귀포쪽에 머물...,['#부영호텔'],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
6381,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,"['#누마루', '#파란하늘', '#방문자리뷰', '#스테이황룡', '#한옥스타그램...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6382,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,"['#정방폭포', '#제주도', '#황사', '#신행', '#신혼여행', '#숙소']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6383,minor_ri,2021-03-30 00:00:00+00:00,381,날씨 엄청 좋았었는데 여행가고싶은 요즘😢💙 13주,[],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6385,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,"['#마지막', '#기록', '#제주', '#여행', '#뚜벅이', '#혼여행', ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


해쉬태그, []값 지워주기

In [291]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

대괄호, # 지우기, 

In [292]:
sukso_merge_list = [] # 비어있는 리스트 만들기 
for txt in sukso_merge['hashtag'] : 
    del_br = txt.strip("[]")
    del_br = del_br.replace("#"," ")
    del_br = del_br.replace(",","")
    del_br = del_br.replace("'","")
    sukso_merge_list.append(del_br) #값 채우기

한옥스테이 - (숙소+숙소추천)  /  (숙소+숙소추천) - 한옥스테이

In [293]:
sukso_merge_series = pd.Series(sukso_merge_list)

In [294]:
sukso_merge_series

0        제주  제주  제주도  숙소  제주숙소  숙소추천  제주숙소추천  제주여행  제주...
1        제주여행  제주도여행  제주도  제주  제주숙소  제주숙소추천  제주독채  제주독...
2                                                    부영호텔
3        캠핑  캠핑장추천  캠핑장  캠핑스타...
4                        이호소락  스테이폴리오  스테이폴리오_서울  북촌  노천탕
                              ...                        
7122     체험  가볼만한곳  제주  서귀포  도자기  도예  귤  감귤  중문  숙소  카...
7123     누마루  파란하늘  방문자리뷰  스테이황룡  한옥스타그램  한옥스테이  한옥게스트...
7124                          정방폭포  제주도  황사  신행  신혼여행  숙소
7125     마지막  기록  제주  여행  뚜벅이  혼여행  마지막피드  즐거웠다  날씨는  ...
7126                                 먹방스타그램  양평  숙소  아침메뉴
Length: 7127, dtype: object

In [295]:
sukso_merge

,id,date,like,text,hashtag,src,target
2,busansomang,2021-06-21 00:00:00+00:00,"2,086","감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...","['#제주', '#제주', '#제주도', '#숙소', '#제주숙소', '#숙소추천'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,zzazang_,2021-06-26 00:00:00+00:00,"3,385",이거 하나보고 예약했던 숙소 🪨🪵🐷🥩🥬🍹💖 . . . 2일,"['#제주여행', '#제주도여행', '#제주도', '#제주', '#제주숙소', '#...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
4,ejiyang,2021-06-09 00:00:00+00:00,"1,785",문의주신 숙소 공유해드려요♥️ 서귀포에 위치한 이에요🌼 이번 제주는 서귀포쪽에 머물...,['#부영호텔'],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
5,so__hyeon,2021-06-26 00:00:00+00:00,"14,145",여기 진짜 미쳤다. 너무너무 좋다는 말 밖에🏕🍃 우리 사이트 바로 앞으로 바람 솔솔...,"['#캠핑', '#캠핑장추천', '#캠핑장', '...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7,stayfolio\n인증됨,2021-06-24 00:00:00+00:00,"1,892","인증됨 작고(釐毫) 소박한 즐거움(小樂)을 더하다, 화려하게 꾸며진 공간은 많다. ...","['#이호소락', '#스테이폴리오', '#스테이폴리오_서울', '#북촌', '#노천탕']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
6377,mhc022012,2021-03-30 00:00:00+00:00,영상,Happy pottery studio Jeju 행복공방 에치비 도자기체험 제주 Ma...,"['#체험', '#가볼만한곳', '#제주', '#서귀포', '#도자기', '#도예'...",NaN,0
6381,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,"['#누마루', '#파란하늘', '#방문자리뷰', '#스테이황룡', '#한옥스타그램...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6382,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,"['#정방폭포', '#제주도', '#황사', '#신행', '#신혼여행', '#숙소']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6385,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,"['#마지막', '#기록', '#제주', '#여행', '#뚜벅이', '#혼여행', ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


숙소 값에 0 주기

In [296]:
sukso_merge_series= sukso_merge_series.str.contains("한옥스테이") ==0
sukso_merge_series= sukso_merge_series[sukso_merge_series] 

In [297]:
sukso_merge

,id,date,like,text,hashtag,src,target
2,busansomang,2021-06-21 00:00:00+00:00,"2,086","감성숙소 모음리스트✈️ 갬성, 힐링 모든 면에서 대박인 제쥬도 존예 숙소 다 모아와...","['#제주', '#제주', '#제주도', '#숙소', '#제주숙소', '#숙소추천'...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
3,zzazang_,2021-06-26 00:00:00+00:00,"3,385",이거 하나보고 예약했던 숙소 🪨🪵🐷🥩🥬🍹💖 . . . 2일,"['#제주여행', '#제주도여행', '#제주도', '#제주', '#제주숙소', '#...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
4,ejiyang,2021-06-09 00:00:00+00:00,"1,785",문의주신 숙소 공유해드려요♥️ 서귀포에 위치한 이에요🌼 이번 제주는 서귀포쪽에 머물...,['#부영호텔'],https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
5,so__hyeon,2021-06-26 00:00:00+00:00,"14,145",여기 진짜 미쳤다. 너무너무 좋다는 말 밖에🏕🍃 우리 사이트 바로 앞으로 바람 솔솔...,"['#캠핑', '#캠핑장추천', '#캠핑장', '...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
7,stayfolio\n인증됨,2021-06-24 00:00:00+00:00,"1,892","인증됨 작고(釐毫) 소박한 즐거움(小樂)을 더하다, 화려하게 꾸며진 공간은 많다. ...","['#이호소락', '#스테이폴리오', '#스테이폴리오_서울', '#북촌', '#노천탕']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...
6377,mhc022012,2021-03-30 00:00:00+00:00,영상,Happy pottery studio Jeju 행복공방 에치비 도자기체험 제주 Ma...,"['#체험', '#가볼만한곳', '#제주', '#서귀포', '#도자기', '#도예'...",NaN,0
6381,seo_yeong_star,2021-03-30 00:00:00+00:00,246,방문자리뷰에 올려주신 후기 사진이예요 파란하늘과 마루방을 너무 이쁘게 담아주셔서 가...,"['#누마루', '#파란하늘', '#방문자리뷰', '#스테이황룡', '#한옥스타그램...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6382,___ju1004,2021-03-30 00:00:00+00:00,24,제주도에서의 하루^^ 제주도도 황사가 장난아님...;;; . . . 13주,"['#정방폭포', '#제주도', '#황사', '#신행', '#신혼여행', '#숙소']",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0
6385,hyejini_vv,2021-03-30 00:00:00+00:00,20,제주 여행 여덟번째기록📝 노잼여행을 유잼여행으로 만들어주신 @youngmi__s2 ...,"['#마지막', '#기록', '#제주', '#여행', '#뚜벅이', '#혼여행', ...",https://scontent-ssn1-1.cdninstagram.com/v/t51...,0


In [298]:
sukso_merge_Series.value_counts()

 사시장청                                                                                                                                                                                                              55
 체험  가볼만한곳  제주  서귀포  도자기  도예  귤  감귤  중문  숙소  카페  바다  성산  풍차  성지  시선  陶器  challenge  Jeju  where  sightseeing  money  work  happy  experience  beautiful  sea  ocean                                                53
 첨성관                                                                                                                                                                                                               52
 보야지  제주  제주도  제주여행  여행  감성여행  여행사진  한달살기  숙소추천  여행기록  제주공항  제주도여행  제주도가볼만한곳  제주도민  제주스냅  뚜벅이여행  제주살이  삼양해수욕장  제주동쪽게스트하우스  제주게하  게스트하우스  소규모게스트하우스  제주게스트하우스  삼양숙소  삼양게스트하우스  동쪽숙소  감성게스트하우스                       45
 경주펜션  경주호텔  경주풀빌라  경주여행  경주숙소  경주여행코스  경주키즈풀빌라  경주숙소추천  경주키즈펜션  경주숙박  경주독채펜션  경주풀빌라펜션  경주애견동반펜션  포항펜션  포항풀빌라  부산펜션  부산풀빌라  울산펜션  울산풀빌라  포항숙소  울

필요없는 칼럼 지우기

In [299]:
index_del2 = sukso_merge[sukso_merge['hashtag'] == '[]'].index
sukso_merge = sukso_merge.drop(index_del2)

In [300]:
len(sukso_merge_list)

7127

In [301]:
len(sukso_merge)

7127

In [302]:
sukso_merge.reset_index(drop = True, inplace = True)

In [303]:
sukso_merge['hashtag'] =sukso_merge_list

In [304]:
sukso_merge['hashtag']

0        제주  제주  제주도  숙소  제주숙소  숙소추천  제주숙소추천  제주여행  제주...
1        제주여행  제주도여행  제주도  제주  제주숙소  제주숙소추천  제주독채  제주독...
2                                                    부영호텔
3        캠핑  캠핑장추천  캠핑장  캠핑스타...
4                        이호소락  스테이폴리오  스테이폴리오_서울  북촌  노천탕
                              ...                        
7122     체험  가볼만한곳  제주  서귀포  도자기  도예  귤  감귤  중문  숙소  카...
7123     누마루  파란하늘  방문자리뷰  스테이황룡  한옥스타그램  한옥스테이  한옥게스트...
7124                          정방폭포  제주도  황사  신행  신혼여행  숙소
7125     마지막  기록  제주  여행  뚜벅이  혼여행  마지막피드  즐거웠다  날씨는  ...
7126                                 먹방스타그램  양평  숙소  아침메뉴
Name: hashtag, Length: 7127, dtype: object

In [305]:
sukso_merge_data= sukso_merge.iloc[:, [4,6]]

In [321]:
hanok_plus_sukso_merge = pd.concat([hanok_data,sukso_merge_data], axis = 0)

In [323]:
hanok_plus_sukso_merge_X = hanok_plus_sukso_merge.iloc[:, 0]
hanok_plus_sukso_merge_y = hanok_plus_sukso_merge.iloc[:, 1]

In [359]:
hanok_plus_sukso_merge.iloc[3000]

hashtag    서촌영락재 대학동기 서촌 한옥스테이 추억 마음은 청춘 daytrip hanok se...
target                                                     1
Name: 3000, dtype: object

## 머신러닝

In [308]:
from sklearn.model_selection import train_test_split

In [326]:
X_train, X_test, y_train, y_test = train_test_split(hanok_plus_sukso_merge_X, # 문제데이터
                                                    hanok_plus_sukso_merge_y,  #정답데이터
                                                   random_state = 3,  # 섞는 방법
                                                   test_size =0.3)

In [327]:
from sklearn.model_selection import cross_val_score

####  BOW(Bag Of Word)
- 단어 토큰화, 단어사전 구축을 통한 수치화

In [328]:
from sklearn.feature_extraction.text import CountVectorizer
testCV = CountVectorizer()

In [329]:
testCV.fit(X_train, y_train)

CountVectorizer()

In [330]:
X_train_CV = testCV.transform(X_train)
X_test_CV = testCV.transform(X_test)

In [331]:
# svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
svm = LinearSVC()

In [332]:
X_train_CV

<7548x22815 sparse matrix of type '<class 'numpy.int64'>'
	with 102744 stored elements in Compressed Sparse Row format>

In [333]:
svm_result = cross_val_score(svm, X_train_CV, y_train, cv=5)
svm_result.mean()

0.9501855972333768

In [336]:
svm.fit(X_train_CV,y_train)

LinearSVC()

In [338]:
svm.predict(X_train_CV)

array([1, 1, 0, ..., 0, 1, 0], dtype=int64)

In [344]:
healing = ["'파리는','파리게스트하우스','힐링숙소','파리여행의달달한동반자','파리민박'"]

In [356]:
test = ["'본채 남원여행 남원숙소 남원 스테이리운 전북남원 감성숙소 춘향제'"]

In [369]:
test2 = ["'비빔밥 콩나물국밥 풍년제과 한옥마을 '"]

In [405]:
test3 = ["'남원 전주'"]

전주데이터가 부족하다는 것을 파악할 수 있음


In [403]:
reviews_transform = testCV.transform(test3) #토큰화, 수치화

In [404]:
svm.predict(reviews_transform)

array([0], dtype=int64)

In [ ]:
kkma =Kkma()

In [ ]:
def myTokenizer(text):
    return kkma.nouns(text)

In [ ]:
tfidf = TfidfVectorizer(tokenizer = myTokenizer)

In [ ]:
tfidf.fit(stay)

In [ ]:
tfidf.vocabulary_

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

In [ ]:
sukso_recommnedation_text['#숙소']